# Practicumopdrachten Week 2.2 (eerste kans)

## Dependencies importeren

Importeer in het codeblok hieronder de packages die worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
N.B.: de 2 reeds geschreven coderegels zorgen ervoor dat eventuele warnings, die code-outputs lelijker maken, uitgezet worden.

In [3]:

import warnings
import numpy as np
import sqlite3
import pandas as pd
warnings.simplefilter('ignore')

Zet het bestand go_sales.sqlite in hetzelfde mapje als dit notebook.

## Databasetabel inlezen

Creëer een databaseconnectie met het bestand go_sales.sqlite.

In [4]:
con = sqlite3.connect("Assets/Week 2/go_sales.sqlite")

Gebruik de onderstaande sql_query om te achterhalen welke databasetabellen in go_sales zitten.

In [57]:
sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
#Vul dit codeblok verder in
def discoverTables() :
    try :
        con = sqlite3.connect("Assets/Week 2/go_sales.sqlite")
        cursor = con.cursor() 
        print("Connected to sqlite.")
        cursor.execute(sql_query)
        records = cursor.fetchall()
        print("Total rows are:  ", len(records))
        for row in records : 
            print(f"    {row}")
    except sqlite3.Error as error:
        print("Failed to read data from sqlite table", error)
    finally:
        if con:
            con.close()
            print("The SQLite connection is now closed.")

discoverTables()

Connected to sqlite.
Total rows are:   14
    ('country',)
    ('order_details',)
    ('order_header',)
    ('order_method',)
    ('product',)
    ('product_line',)
    ('product_type',)
    ('retailer_site',)
    ('return_reason',)
    ('returned_item',)
    ('sales_branch',)
    ('sales_staff',)
    ('SALES_TARGETData',)
    ('sqlite_sequence',)
The SQLite connection is now closed.


Gebruik de gecreëerde databaseconnectie om de resultaten van de volgende query in een DataFrame te zetten:<br>
*SELECT * FROM sales_staff* 

In [58]:
sql_query = "SELECT * FROM sales_staff;"
global original_result

def dataframe() : 
    try : 
        con = sqlite3.connect("Assets/Week 2/go_sales.sqlite")
        print("Connected to sqlite.")
        result = pd.read_sql_query(sql_query,con)
        print(result)
        global original_result
        original_result = result
    except sqlite3.Error as error: 
        print("Failed to read data from sqlite table", error)
    finally : 
        if con:
            con.close()
            print("The SQLite connection is now closed.")

dataframe()

Connected to sqlite.
    SALES_STAFF_CODE FIRST_NAME   LAST_NAME                   POSITION_EN  \
0                100     Tuomas  Savolainen  Level 2 Sales Representative   
1                101   Chang-ho         Kim  Level 2 Sales Representative   
2                102    Jung-ho        Choi  Level 3 Sales Representative   
3                103  Kwei-Ping         Kao  Level 2 Sales Representative   
4                104  Chin-Tsai        Fang  Level 3 Sales Representative   
..               ...        ...         ...                           ...   
97                95      Tomás    Iglesias  Level 2 Sales Representative   
98                96     Bayard       Lopes                Branch Manager   
99                97     Viviam      Barros  Level 3 Sales Representative   
100               98    Eduardo   Guimarães  Level 2 Sales Representative   
101               99     Helena    Lindholm  Level 3 Sales Representative   

              WORK_PHONE EXTENSION                  FA

## DataFrames uitsplitsen en opbouwen met Series

De volgende 5 kolommen hebben betrekking op de contactdetails van elke medewerker in dit DataFrame:
- SALES_STAFF_CODE
- WORK_PHONE
- EXTENSION
- FAX
- EMAIL

Maak van elk van deze 5 kolommen een serie.

In [7]:
sql_query = "SELECT * FROM sales_staff;"
global sales_staff_column 
global work_phone_column
global extension_column
global fax_column
global email_column


def go_sales_series() : 
   try : 
        con = sqlite3.connect("Assets/Week 2/go_sales.sqlite")
        print("Connected to sqlite.")
        result =  pd.read_sql_query(sql_query,con)

        global sales_staff_column 
        sales_staff_column = result["SALES_STAFF_CODE"]

        global work_phone_column
        work_phone_column = result["WORK_PHONE"]

        global extension_column
        extension_column = result["EXTENSION"]

        global fax_column
        fax_column = result["FAX"]

        global email_column
        email_column = result["EMAIL"]

        db_series = (sales_staff_column,work_phone_column,extension_column,fax_column,email_column)
        #print(db_series)

        #print(result)
        # print(sales_staff_column)
   except sqlite3.Error as error: 
       print("Failed to read data from sqlite table", error)
   finally : 
        if con:
            con.close()
            print("The SQLite connection is now closed.")

go_sales_series()

#print(sales_staff_column)
#print(work_phone_column)
#print(extension_column)
#print(fax_column)
#print(email_column)

Connected to sqlite.
The SQLite connection is now closed.


Zet allevijf gecreëerde series als kolommen naast elkaar in een DataFrame (*contact_details*). Gebruik pd.concat(...)<br>
Hulpvraag: welke waarde geef je aan de axis-parameter?

In [8]:
# print(sales_staff_column)
# print(work_phone_column)
# print(extension_column)
# print(fax_column)
# print(email_column)



contact_details = pd.concat([sales_staff_column,work_phone_column,extension_column,fax_column,email_column], axis=1)
print(contact_details)

    SALES_STAFF_CODE           WORK_PHONE EXTENSION                  FAX  \
0                100  +358(0)17 - 433 127       825  +358(0)17 - 433 129   
1                101        82-2-778-6587      1228        82-2-778-6586   
2                102        82-2-778-6587      2641        82-2-778-6586   
3                103      886-2-2975-1988      1541      886-2-2989-4978   
4                104      886-2-2975-1988      2311      886-2-2989-4978   
..               ...                  ...       ...                  ...   
97                95      +35 94 322 3540       325      +35 91 548 1637   
98                96    +55 (11) 344-4444      3490    +55 (11) 333-2223   
99                97    +55 (11) 344-4444      3364    +55 (11) 333-2223   
100               98    +55 (11) 344-4444      1442    +55 (11) 333-2223   
101               99  +358(0)17 - 433 127       489  +358(0)17 - 433 129   

                       EMAIL  
0    TSavolainen@grtd123.com  
1           CKim@grtd123.

## Series en DataFrames maken vanuit lists en dictionaries

Met .head(*getal*) kan je de bovenste *getal* rijen van een tabel selecteren.<br>
Selecteer op deze manier de bovenste 5 rijen van *contact_details*.<br>
Sla dit resultaat op in een nieuw DataFrame.

In [32]:
# print(contact_details.head(5))
top5_contactdetails = pd.DataFrame(contact_details.head(5))
print(top5_contactdetails)

  SALES_STAFF_CODE           WORK_PHONE EXTENSION                  FAX  \
0              100  +358(0)17 - 433 127       825  +358(0)17 - 433 129   
1              101        82-2-778-6587      1228        82-2-778-6586   
2              102        82-2-778-6587      2641        82-2-778-6586   
3              103      886-2-2975-1988      1541      886-2-2989-4978   
4              104      886-2-2975-1988      2311      886-2-2989-4978   

                     EMAIL  
0  TSavolainen@grtd123.com  
1         CKim@grtd123.com  
2        JChoi@grtd123.com  
3        KKwao@grtd123.com  
4         CFon@grtd123.com  


Aan deze 10 rijen met contactdetails willen we 3 kolommen toevoegen: 'FIRST_LANGUAGE', 'SECOND_LANGUAGE' & 'THIRD_LANGUAGE'.<br>
Iedereens 'First Language' is Engels, afgekort 'EN'. Maak een lijst waarin 5x 'EN' staat.<br>
Converteer deze lijst vervolgens naar een serie en voeg deze horizontaal samen met het resultaat van de vorige opdracht.<br>
Vergeet niet een passende naam te geven aan de nieuw ontstane kolom.

In [33]:
language_column = pd.Series(["EN","EN","EN","EN","EN"], name="FIRST_LANGUAGE")
truncated_Contact = pd.concat([top5_contactdetails,language_column], axis=1)
print(truncated_Contact)

  SALES_STAFF_CODE           WORK_PHONE EXTENSION                  FAX  \
0              100  +358(0)17 - 433 127       825  +358(0)17 - 433 129   
1              101        82-2-778-6587      1228        82-2-778-6586   
2              102        82-2-778-6587      2641        82-2-778-6586   
3              103      886-2-2975-1988      1541      886-2-2989-4978   
4              104      886-2-2975-1988      2311      886-2-2989-4978   

                     EMAIL FIRST_LANGUAGE  
0  TSavolainen@grtd123.com             EN  
1         CKim@grtd123.com             EN  
2        JChoi@grtd123.com             EN  
3        KKwao@grtd123.com             EN  
4         CFon@grtd123.com             EN  


Maak nu de tweede kolom ('SECOND_LANGUAGE'). Maak daarvoor een dictionary, met daarin...
- Als keys: alle indexen uit het resultaat van het vorige codeblok.
- Als values: bij de eerste 3 elementen 'FR' (Frankrijk), bij de laatste 2 'DE' (Duitsland).

Maak vervolgens ook hier weer een serie van en voeg ook deze weer horizontaal samen met het rsultaat van de vorige opdracht.<br>
Vergeet niet een passende naam te geven aan de nieuw ontstane kolom.

In [34]:
second_language_column = pd.Series({0:"FR", 1:"FR",2:"FR",3:"DE",4:"DE"}, name="SECOND_LANGUAGE")
truncated_Contact = pd.concat([truncated_Contact, second_language_column], axis=1)
print(truncated_Contact)

  SALES_STAFF_CODE           WORK_PHONE EXTENSION                  FAX  \
0              100  +358(0)17 - 433 127       825  +358(0)17 - 433 129   
1              101        82-2-778-6587      1228        82-2-778-6586   
2              102        82-2-778-6587      2641        82-2-778-6586   
3              103      886-2-2975-1988      1541      886-2-2989-4978   
4              104      886-2-2975-1988      2311      886-2-2989-4978   

                     EMAIL FIRST_LANGUAGE SECOND_LANGUAGE  
0  TSavolainen@grtd123.com             EN              FR  
1         CKim@grtd123.com             EN              FR  
2        JChoi@grtd123.com             EN              FR  
3        KKwao@grtd123.com             EN              DE  
4         CFon@grtd123.com             EN              DE  


Maak ten slotte de derde kolom ('THIRD_LANGUAGE') door een dictionary te maken met daarin...
- Als key: de naam van de nieuwe kolom. Zie je het verschil qua keys met de vorige opdracht?
- Als waarde: een lijst met daarin 'NL', 'NL', 'JPN', 'JPN', 'KOR'.

Converteer deze dictionary nu naar een DataFrame en voeg deze horizontaal samen met het resultaat van de vorige opdracht.<br>
Waarom hoef je hierna de nieuw ontstane kolom geen passende naam meer te geven?

In [41]:
third_language_dict = {"THIRD_LANGUAGE":["NL","NL","JPN","JPN","KOR"]}
third_language_column = pd.DataFrame.from_dict(third_language_dict)
df_third_lang = pd.DataFrame(third_language_column)
# print(df_third_lang)
truncated_Contact = pd.concat([truncated_Contact,df_third_lang], axis=1)
print(truncated_Contact)

  SALES_STAFF_CODE           WORK_PHONE EXTENSION                  FAX  \
0              100  +358(0)17 - 433 127       825  +358(0)17 - 433 129   
1              101        82-2-778-6587      1228        82-2-778-6586   
2              102        82-2-778-6587      2641        82-2-778-6586   
3              103      886-2-2975-1988      1541      886-2-2989-4978   
4              104      886-2-2975-1988      2311      886-2-2989-4978   

                     EMAIL FIRST_LANGUAGE SECOND_LANGUAGE THIRD_LANGUAGE  \
0  TSavolainen@grtd123.com             EN              FR             NL   
1         CKim@grtd123.com             EN              FR             NL   
2        JChoi@grtd123.com             EN              FR            JPN   
3        KKwao@grtd123.com             EN              DE            JPN   
4         CFon@grtd123.com             EN              DE            KOR   

  THIRD_LANGUAGE  
0             NL  
1             NL  
2            JPN  
3            JPN  
4  

## Data toevoegen

### Rijen

Gebruik de originele databasetabel *sales_staff*.<br>
Voeg een extra rij toe met eigen invulling. Zorg ervoor dat de index netjes doorloopt.<br>
Hulpvraag: welke waarde geef je nu aan axis?

In [53]:

# print(original_result)
DT1 = original_result

new_row = {
    "SALES_STAFF_CODE":[999],
    "FIRST_NAME":["Nieuw voornaam"],
    "LAST_NAME":["Niewe achternaam"],
    "POSITION_EN":["General Manager"], 
    "WORK_PHONE":["0612345678"],
    "EXTENSION":["9876"],
    "FAX":["+(44) 181 5030282"], 
    "MAIL":["HWilloughby@grtd123.com"],
    "DATE_HIRED":["10-2-2016"], 
    "SALES_BRANCH_CODE":["39"], 
    "TRIAL888":["T"]  }
new_pd = pd.concat([DT1, pd.DataFrame(new_row)], ignore_index=True)
print(new_pd)


    SALES_STAFF_CODE      FIRST_NAME         LAST_NAME  \
0                100          Tuomas        Savolainen   
1                101        Chang-ho               Kim   
2                102         Jung-ho              Choi   
3                103       Kwei-Ping               Kao   
4                104       Chin-Tsai              Fang   
..               ...             ...               ...   
98                96          Bayard             Lopes   
99                97          Viviam            Barros   
100               98         Eduardo         Guimarães   
101               99          Helena          Lindholm   
102              999  Nieuw voornaam  Niewe achternaam   

                      POSITION_EN           WORK_PHONE EXTENSION  \
0    Level 2 Sales Representative  +358(0)17 - 433 127       825   
1    Level 2 Sales Representative        82-2-778-6587      1228   
2    Level 3 Sales Representative        82-2-778-6587      2641   
3    Level 2 Sales Representati

### Kolommen

Voeg een kolom *FULL_NAME* toe die de waarden van *FIRST_NAME* en *LAST_NAME* achter elkaar zet, gescheiden door een spatie.

In [55]:
joincolumn = original_result
joincolumn["FULL_NAME"] = joincolumn[["FIRST_NAME", "LAST_NAME"]].agg( " ".join, axis=1)
print(original_result)

    SALES_STAFF_CODE FIRST_NAME   LAST_NAME                   POSITION_EN  \
0                100     Tuomas  Savolainen  Level 2 Sales Representative   
1                101   Chang-ho         Kim  Level 2 Sales Representative   
2                102    Jung-ho        Choi  Level 3 Sales Representative   
3                103  Kwei-Ping         Kao  Level 2 Sales Representative   
4                104  Chin-Tsai        Fang  Level 3 Sales Representative   
..               ...        ...         ...                           ...   
97                95      Tomás    Iglesias  Level 2 Sales Representative   
98                96     Bayard       Lopes                Branch Manager   
99                97     Viviam      Barros  Level 3 Sales Representative   
100               98    Eduardo   Guimarães  Level 2 Sales Representative   
101               99     Helena    Lindholm  Level 3 Sales Representative   

              WORK_PHONE EXTENSION                  FAX  \
0    +358(0)17 -

## Data wijzigen

### Datatypen

Door het attribuut .dtypes van een DataFrame op te vragen krijg je een serie die per kolom het datatype weergeeft. Doe dit bij de originele databasetabel *sales_staff*

In [56]:
print(original_result.dtypes)

SALES_STAFF_CODE     object
FIRST_NAME           object
LAST_NAME            object
POSITION_EN          object
WORK_PHONE           object
EXTENSION            object
FAX                  object
EMAIL                object
DATE_HIRED           object
SALES_BRANCH_CODE    object
TRIAL888             object
FULL_NAME            object
dtype: object


Hier valt op dat elke kolom het datatype 'object' heeft: Python leest dus alles als string. Wiskundige operaties zijn hierdoor niet mogelijk.<br>
We kunnen proberen om kolommen met getallen, bijvoorbeeld de 'extension', te converteren naar een int. Zie onderstaande code.

In [61]:
convertInt = original_result
# convertInt['EXTENSION'] = convertInt['EXTENSION'].astype(int)
convertInt = convertInt.convert_dtypes
print(convertInt.dtypes)

AttributeError: 'function' object has no attribute 'dtypes'

Dit lukt echter niet, omdat er in de kolom 'EXTENSION' lege waarden zitten die niet geconverteerd kunnen worden naar een int.<br>
Wél kan je deze naar een float converteren, zie onderstaande code:

In [64]:
convertInt = original_result
convertInt['EXTENSION'] = convertInt['EXTENSION'].astype(float)
convertInt.dtypes

SALES_STAFF_CODE      object
FIRST_NAME            object
LAST_NAME             object
POSITION_EN           object
WORK_PHONE            object
EXTENSION            float64
FAX                   object
EMAIL                 object
DATE_HIRED            object
SALES_BRANCH_CODE     object
TRIAL888              object
dtype: object

Als we in de rij van 'EXTENSION' kijken zien we dat de conversie van het datatype nu is gelukt.<br>
Dit is <b>randvoorwaardelijk</b> voor het uitvoeren van wiskundige operaties.<br>

### Rijen

Zorg er nu voor dat bij alle extensions 1 wordt opgeteld.

In [65]:
convertInt["EXTENSION"] += 1
print(convertInt)

    SALES_STAFF_CODE FIRST_NAME   LAST_NAME                   POSITION_EN  \
0                100     Tuomas  Savolainen  Level 2 Sales Representative   
1                101   Chang-ho         Kim  Level 2 Sales Representative   
2                102    Jung-ho        Choi  Level 3 Sales Representative   
3                103  Kwei-Ping         Kao  Level 2 Sales Representative   
4                104  Chin-Tsai        Fang  Level 3 Sales Representative   
..               ...        ...         ...                           ...   
97                95      Tomás    Iglesias  Level 2 Sales Representative   
98                96     Bayard       Lopes                Branch Manager   
99                97     Viviam      Barros  Level 3 Sales Representative   
100               98    Eduardo   Guimarães  Level 2 Sales Representative   
101               99     Helena    Lindholm  Level 3 Sales Representative   

              WORK_PHONE  EXTENSION                  FAX  \
0    +358(0)17 

Elke 'Branch Manager' wordt nu 'General Manager'. Schrijf code zodat deze wijziging doorgevoerd wordt in het DataFrame.

In [69]:
convertInt.replace("Branch Manager", "General Manager")
print(convertInt)

    SALES_STAFF_CODE FIRST_NAME   LAST_NAME                   POSITION_EN  \
0                100     Tuomas  Savolainen  Level 2 Sales Representative   
1                101   Chang-ho         Kim  Level 2 Sales Representative   
2                102    Jung-ho        Choi  Level 3 Sales Representative   
3                103  Kwei-Ping         Kao  Level 2 Sales Representative   
4                104  Chin-Tsai        Fang  Level 3 Sales Representative   
..               ...        ...         ...                           ...   
97                95      Tomás    Iglesias  Level 2 Sales Representative   
98                96     Bayard       Lopes                Branch Manager   
99                97     Viviam      Barros  Level 3 Sales Representative   
100               98    Eduardo   Guimarães  Level 2 Sales Representative   
101               99     Helena    Lindholm  Level 3 Sales Representative   

              WORK_PHONE  EXTENSION                  FAX  \
0    +358(0)17 

### Kolommen

Verander de kolomnaam 'POSITION_EN' naar 'POSITION'.

## Data verwijderen

### Rijen

De medewerkers op indexen 99, 100 en 101 hebben helaas ontslag genomen.<br>
Verwijder de bijbehorende rijen uit het DataFrame. Gebruik slechts één keer de .drop()-methode.

### Kolommen

Faxen zijn inmiddels ouderwets: niemand gebruikt zijn/haar faxnummer nog.<br>
Verwijder de bijbehorende kolom uit het DataFrame.